In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
import time

In [9]:
ept = pd.read_excel("ept_all.xlsx")
ept.head()

,ISO,Tanggal,Waktu,Ruangan,Pengawas,ID_Peserta,Hadir,NamaPeserta,TempatLahir,TanggalLahir,...,Versi,Kegiatan,Tahun,L1,G1,R1,L2,G2,R2,TotalSkor
0,567,2023-11-17,13.00 - 15.00,Lab,Indah Anjar Reski,15059,True,ABDIL FIRMANSYAH,PONTIANAK,2005-08-02 00:00:00,...,3,Semester Awal,2023,17.0,10.0,13.0,41.0,33.0,33.0,357.0
1,567,2023-11-17,13.00 - 15.00,Lab,Indah Anjar Reski,15060,True,AMANDA MILING ROSINA,KAPUAS HULU,2005-03-03 00:00:00,...,4,Semester Awal,2023,11.0,6.0,17.0,36.0,28.0,37.0,337.0
2,567,2023-11-17,13.00 - 15.00,Lab,Indah Anjar Reski,15061,True,ANDRE RESTU SAFUTRA,PONTIANAK,2004-12-09 00:00:00,...,5,Semester Awal,2023,13.0,8.0,12.0,38.0,30.0,32.0,333.0
3,567,2023-11-17,13.00 - 15.00,Lab,Indah Anjar Reski,15062,True,ANDRIANI,PONTIANAK,2004-09-16 00:00:00,...,6,Semester Awal,2023,16.0,11.0,12.0,41.0,34.0,32.0,357.0
4,567,2023-11-17,13.00 - 15.00,Lab,Indah Anjar Reski,15063,True,REDITA PUTRI SHABILA,PONTIANAK,2005-11-16 00:00:00,...,7,Semester Awal,2023,28.0,19.0,18.0,48.0,42.0,38.0,427.0


In [17]:
def appendGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name('key.json', scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    existing_iso_values = worksheet.col_values(worksheet.find('ISO').col)  
    existing_iso_values = existing_iso_values[1:] 
    
    # Convert to set for faster comparison
    existing_iso_set = set(existing_iso_values)

    # Get unique ISO values from the DataFrame
    new_iso_values = set(df['ISO'].astype(str).unique())
    
    # Check for duplicates
    duplicates = existing_iso_set.intersection(new_iso_values)
    if duplicates:
        raise ValueError(f"Duplicate ISO values found: {duplicates}. Aborting the operation.")
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be appended
    data = df.values.tolist()
    
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        worksheet.append_rows(batch, value_input_option='RAW')
        time.sleep(1)

    


    

appendGsheet(df=ept, dari=585, sampai=587)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\460031289.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\460031289.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\460031289.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [15]:
def updateGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = Credentials.from_service_account_file('key.json', scopes=scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be update
    data = df.values.tolist()
    
    # Update data in batches to reduce API calls
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        cell_range = f'A{i+15272}:AD{i+15272+len(batch)-1}'  # Adjust the range as needed
        worksheet.update(cell_range, batch)
        time.sleep(1)  # Add delay to avoid hitting rate limits
    
     


updateGsheet(df=ept, dari=581, sampai=584)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\2487365677.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\2487365677.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22320\2487365677.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]